# Dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from config import password

import requests
import time
from datetime import datetime

from scipy.stats import linregress

# Import API key
from config import weather_api_key

## DataFrames Related to COVID-19

In [2]:
# Reading csv files for COVID-19(Cases, Hospitalizations, Death) & NYC Boroughs

bronx_covid_data = "data/covid/Bronx-data-GHyLp.csv"
brooklyn_covid_data = "data/covid/Brooklyn-data-Q7Zjo.csv"
manhattan_covid_data = "data/covid/Manhattan-data-rqvAu.csv"
queens_covid_data = "data/covid/Queens-data-HXuvT.csv"
staten_island_covid_data = "data/covid/Staten Island-data-u1Bfw.csv"

bronx_df = pd.read_csv(bronx_covid_data)
brooklyn_df = pd.read_csv(brooklyn_covid_data)
manhattan_df = pd.read_csv(manhattan_covid_data)
queens_df = pd.read_csv(queens_covid_data)
staten_island_df = pd.read_csv(staten_island_covid_data)

bronx_df.dtypes

DATE_OF_INTEREST    object
Cases                int64
Hospitalizations     int64
Deaths               int64
dtype: object

# Adding Borough Columns to the datafarmes

In [3]:
# Adding Borough Columns to the datafarmes
bronx_df['Borough'] = 'BRONX'
brooklyn_df['Borough'] = 'BROOKLYN'
manhattan_df['Borough'] = 'MANHATTAN'
queens_df['Borough'] = 'QUEENS'
staten_island_df['Borough'] = 'STATEN ISLAND'

# Merging all 5 borough DataFrames into one
frames = [bronx_df, brooklyn_df, manhattan_df, queens_df, staten_island_df]
result_covid = pd.concat(frames).reset_index()


covid_final_df=result_covid.drop(columns=['index'])
covid_final_df = covid_final_df.rename(columns={'DATE_OF_INTEREST':'Date'})

covid_final_df.tail()

,Date,Cases,Hospitalizations,Deaths,Borough
470,05/29/2020,15,0,1,STATEN ISLAND
471,05/30/2020,9,0,0,STATEN ISLAND
472,05/31/2020,6,0,1,STATEN ISLAND
473,06/01/2020,2,0,0,STATEN ISLAND
474,06/02/2020,0,0,0,STATEN ISLAND


In [4]:
covid = covid_final_df["Borough"].unique()
covid

array(['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'],
      dtype=object)

# Getting all lan, lat of the boroughs

In [5]:
# Setting the base URL
url = "http://api.openweathermap.org/data/2.5/weather?"

# Using Imperial as Unit for Fahrenheit 
units = "imperial"

# Building partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [6]:
# Setting up lists to hold reponse info
lat = []
lng = []

# Loop through the list of cities and perform a request for data on each
for index, city in enumerate(covid):
            
    # Filling the Lists if the city was found on the URL
    try:
        response = requests.get(query_url + city).json()
        time.sleep(1)
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])

    except:

        pass


In [7]:
borough_df = pd.DataFrame({'Borough': covid,
                           'Latitude': lat,
                           'Longitude': lng
                          })

In [8]:
covid_final_df = covid_final_df.merge(borough_df, on= "Borough")
covid_final_df

,Date,Cases,Hospitalizations,Deaths,Borough,Latitude,Longitude
0,02/29/2020,0,4,0,BRONX,40.83,-73.92
1,03/01/2020,1,1,0,BRONX,40.83,-73.92
2,03/02/2020,0,9,0,BRONX,40.83,-73.92
3,03/03/2020,1,7,0,BRONX,40.83,-73.92
4,03/04/2020,0,6,0,BRONX,40.83,-73.92
...,...,...,...,...,...,...,...
470,05/29/2020,15,0,1,STATEN ISLAND,40.58,-74.15
471,05/30/2020,9,0,0,STATEN ISLAND,40.58,-74.15
472,05/31/2020,6,0,1,STATEN ISLAND,40.58,-74.15
473,06/01/2020,2,0,0,STATEN ISLAND,40.58,-74.15


# Creating a DataFrame for Total Number of COVIS-19 Cases, Hospitalizations & Death

In [9]:
# Creating a DataFrame for Total Number of COVIS-19 Cases, Hospitalizations & Death

covid_total_df = pd.DataFrame()

covid_total_df['Date'] = bronx_df['DATE_OF_INTEREST']

covid_total_df['TotalCases']=""
covid_total_df['TotalHospitalizations']=""
covid_total_df['TotalDeaths']=""
covid_total_df

,Date,TotalCases,TotalHospitalizations,TotalDeaths
0,02/29/2020,,,
1,03/01/2020,,,
2,03/02/2020,,,
3,03/03/2020,,,
4,03/04/2020,,,
...,...,...,...,...
90,05/29/2020,,,
91,05/30/2020,,,
92,05/31/2020,,,
93,06/01/2020,,,


In [10]:
# Filling the columns

for i in range (len(covid_total_df)):
    covid_total_df.loc[i,'TotalCases'] = bronx_df.iloc[i,1]+brooklyn_df.iloc[i,1]+staten_island_df.iloc[i,1]+manhattan_df.iloc[i,1]+queens_df.iloc[i,1]
    covid_total_df.loc[i,'TotalHospitalizations'] = bronx_df.iloc[i,2]+brooklyn_df.iloc[i,2]+staten_island_df.iloc[i,2]+manhattan_df.iloc[i,2]+queens_df.iloc[i,2]
    covid_total_df.loc[i,'TotalDeaths'] = bronx_df.iloc[i,3]+brooklyn_df.iloc[i,3]+staten_island_df.iloc[i,3]+manhattan_df.iloc[i,3]+queens_df.iloc[i,3]

covid_total_df['TotalCases'] = covid_total_df['TotalCases'].astype('int32')
covid_total_df['TotalHospitalizations'] = covid_total_df['TotalHospitalizations'].astype('int32')
covid_total_df['TotalDeaths'] = covid_total_df['TotalDeaths'].astype('int32')

covid_total_df.head()

,Date,TotalCases,TotalHospitalizations,TotalDeaths
0,02/29/2020,1,12,0
1,03/01/2020,1,4,0
2,03/02/2020,0,22,0
3,03/03/2020,2,19,0
4,03/04/2020,5,21,0


In [11]:
# Finding the start Date of COVID-19 & Latest Date

start_date = covid_final_df.iloc[0,0]

latest_date = covid_final_df.iloc[-1,0]

print('Start Date: '+start_date+ ' & Latest Date: '+ latest_date)

Start Date: 02/29/2020 & Latest Date: 06/02/2020


## DataFrames Related to CRIMES

In [12]:
# Reading the CSV file for New York City Police Department

nypd = "data/crimes/NYPD.csv"
nypd_data = pd.read_csv(nypd)
nypd_data.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,45975594,04/11/2020 01:50:25 AM,04/11/2020 02:27:47 AM,NYPD,New York City Police Department,Non-Emergency Police Matter,Social Distancing,Residential Building/House,10025.0,222 WEST 104 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.799478,-73.967504,"(40.799477899696285, -73.96750426182348)"
1,45975654,04/11/2020 12:14:28 AM,04/11/2020 02:17:36 AM,NYPD,New York City Police Department,Non-Emergency Police Matter,Social Distancing,NaN,10451.0,304 EAST 156 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.821747,-73.919234,"(40.82174682885639, -73.9192338660681)"
2,45975989,04/10/2020 06:31:17 PM,04/11/2020 05:48:01 AM,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11419.0,107-11 129 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.686233,-73.814735,"(40.68623317590883, -73.81473500807529)"
3,45975997,04/11/2020 12:31:45 AM,04/11/2020 07:57:45 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11102.0,25-57 31 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.769635,-73.918557,"(40.76963491074108, -73.91855696912114)"
4,45976007,04/10/2020 11:31:56 PM,04/11/2020 03:27:47 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10458.0,340 EAST 184 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.858098,-73.895670,"(40.858098079528844, -73.89567035668057)"


In [13]:
# Checking the columns for costomization

nypd_data.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Action Updated Date', 'Community Board', 'Borough',
       'X Coordinate (State Plane)', 'Y Coordinate (State Plane)',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [14]:
# Selecting the required columns for creating a new dataFrame

nypd_columns = ['Created Date', 'Complaint Type', 'Descriptor', 'Location Type','Incident Address','City','Borough','Latitude', 'Longitude']

nypd_df = nypd_data[nypd_columns].copy()

nypd_df

,Created Date,Complaint Type,Descriptor,Location Type,Incident Address,City,Borough,Latitude,Longitude
0,04/11/2020 01:50:25 AM,Non-Emergency Police Matter,Social Distancing,Residential Building/House,222 WEST 104 STREET,NEW YORK,MANHATTAN,40.799478,-73.967504
1,04/11/2020 12:14:28 AM,Non-Emergency Police Matter,Social Distancing,NaN,304 EAST 156 STREET,BRONX,BRONX,40.821747,-73.919234
2,04/10/2020 06:31:17 PM,Noise - Residential,Banging/Pounding,Residential Building/House,107-11 129 STREET,SOUTH RICHMOND HILL,QUEENS,40.686233,-73.814735
3,04/11/2020 12:31:45 AM,Noise - Residential,Loud Music/Party,Residential Building/House,25-57 31 STREET,ASTORIA,QUEENS,40.769635,-73.918557
4,04/10/2020 11:31:56 PM,Noise - Residential,Loud Music/Party,Residential Building/House,340 EAST 184 STREET,BRONX,BRONX,40.858098,-73.895670
...,...,...,...,...,...,...,...,...,...
320192,04/29/2020 02:50:10 PM,Blocked Driveway,Partial Access,Street/Sidewalk,3600 CORLEAR AVENUE,BRONX,BRONX,40.884454,-73.903119
320193,04/29/2020 02:50:13 PM,Blocked Driveway,Partial Access,Street/Sidewalk,3600 CORLEAR AVENUE,BRONX,BRONX,40.884454,-73.903119
320194,04/29/2020 02:50:29 PM,Blocked Driveway,No Access,Street/Sidewalk,76-19 91 AVENUE,WOODHAVEN,QUEENS,40.686247,-73.864151
320195,04/29/2020 02:50:26 PM,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,220 STEWART AVENUE,BROOKLYN,BROOKLYN,40.713264,-73.927727


In [15]:
# Removing Unspecified from Borough Column
nypd_df = nypd_df[nypd_df['Borough'] != "Unspecified"]

# Checking for being sure that Boroughs are Correct
nypd_df['Borough'].unique()

array(['MANHATTAN', 'BRONX', 'QUEENS', 'BROOKLYN', 'STATEN ISLAND'],
      dtype=object)

In [16]:
# Sorting the NYPD DataFrame by Dates
nypd_df= nypd_df.sort_values(['Created Date'])

nypd_df

,Created Date,Complaint Type,Descriptor,Location Type,Incident Address,City,Borough,Latitude,Longitude
134381,02/01/2020 01:00:00 AM,Noise - Commercial,Loud Music/Party,Store/Commercial,110-46 SUTPHIN BOULEVARD,JAMAICA,QUEENS,40.689070,-73.795951
134287,02/01/2020 01:00:16 AM,Noise - Residential,Banging/Pounding,Residential Building/House,200 BENNETT AVENUE,NEW YORK,MANHATTAN,40.856471,-73.933865
143083,02/01/2020 01:00:16 PM,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,425 EAST 14 STREET,NEW YORK,MANHATTAN,40.731055,-73.981783
141734,02/01/2020 01:00:19 PM,Illegal Parking,Blocked Hydrant,Street/Sidewalk,1485 HOE AVENUE,BRONX,BRONX,40.832816,-73.889368
134360,02/01/2020 01:00:48 AM,Noise - Residential,Banging/Pounding,Residential Building/House,1768 77 STREET,BROOKLYN,BROOKLYN,40.612740,-73.998055
...,...,...,...,...,...,...,...,...,...
319095,06/02/2020 12:57:54 AM,Illegal Parking,Blocked Hydrant,Street/Sidewalk,34 AVENUE,NaN,QUEENS,40.751932,-73.902156
318261,06/02/2020 12:58:50 AM,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,488 WEST 130 STREET,NEW YORK,MANHATTAN,40.815552,-73.954126
319569,06/02/2020 12:58:57 AM,Noise - Residential,Loud Music/Party,Residential Building/House,95 OLD BROADWAY,NEW YORK,MANHATTAN,40.818146,-73.955374
318854,06/02/2020 12:59:13 AM,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,37-37 83 STREET,JACKSON HEIGHTS,QUEENS,40.749505,-73.883310


In [17]:
# Removing information of before the Start Date of COVID-19 from NYPD
nypd_df = nypd_df.loc[nypd_df['Created Date'] > start_date]

# Removing NA Rows
nypd_final_df = nypd_df.dropna()

nypd_final_df

,Created Date,Complaint Type,Descriptor,Location Type,Incident Address,City,Borough,Latitude,Longitude
22915,02/29/2020 01:00:47 AM,Illegal Parking,Blocked Hydrant,Street/Sidewalk,1550 EAST 96 STREET,BROOKLYN,BROOKLYN,40.635450,-73.892913
25956,02/29/2020 01:01:36 PM,Illegal Parking,Blocked Bike Lane,Street/Sidewalk,720 BROADWAY,NEW YORK,MANHATTAN,40.729203,-73.993567
25421,02/29/2020 01:01:56 AM,Noise - Residential,Loud Music/Party,Residential Building/House,772 DEAN STREET,BROOKLYN,BROOKLYN,40.679474,-73.964574
26193,02/29/2020 01:02:10 AM,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,131 CHRYSTIE STREET,NEW YORK,MANHATTAN,40.719369,-73.993218
24203,02/29/2020 01:02:39 PM,Noise - Residential,Loud Music/Party,Residential Building/House,1973 81 STREET,BROOKLYN,BROOKLYN,40.607898,-73.996035
...,...,...,...,...,...,...,...,...,...
318972,06/02/2020 12:57:49 AM,Non-Emergency Police Matter,Social Distancing,Store/Commercial,45-20 GREENPOINT AVENUE,SUNNYSIDE,QUEENS,40.741936,-73.919623
318261,06/02/2020 12:58:50 AM,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,488 WEST 130 STREET,NEW YORK,MANHATTAN,40.815552,-73.954126
319569,06/02/2020 12:58:57 AM,Noise - Residential,Loud Music/Party,Residential Building/House,95 OLD BROADWAY,NEW YORK,MANHATTAN,40.818146,-73.955374
318854,06/02/2020 12:59:13 AM,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,37-37 83 STREET,JACKSON HEIGHTS,QUEENS,40.749505,-73.883310


In [18]:
# Removing Time from Date Column
nypd_final_df['Created Date']= nypd_final_df['Created Date'].str.split(" ", n=1, expand= True)

# Renaming the column
nypd_final_df= nypd_final_df.rename(columns={'Created Date': 'Date',
                                            'Complaint Type':'ComplaintType',
                                            'Location Type': 'locationType',
                                            'Incident Address': 'incidentAddress'})

nypd_final_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,ComplaintType,Descriptor,locationType,incidentAddress,City,Borough,Latitude,Longitude
22915,02/29/2020,Illegal Parking,Blocked Hydrant,Street/Sidewalk,1550 EAST 96 STREET,BROOKLYN,BROOKLYN,40.635450,-73.892913
25956,02/29/2020,Illegal Parking,Blocked Bike Lane,Street/Sidewalk,720 BROADWAY,NEW YORK,MANHATTAN,40.729203,-73.993567
25421,02/29/2020,Noise - Residential,Loud Music/Party,Residential Building/House,772 DEAN STREET,BROOKLYN,BROOKLYN,40.679474,-73.964574
26193,02/29/2020,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,131 CHRYSTIE STREET,NEW YORK,MANHATTAN,40.719369,-73.993218
24203,02/29/2020,Noise - Residential,Loud Music/Party,Residential Building/House,1973 81 STREET,BROOKLYN,BROOKLYN,40.607898,-73.996035
...,...,...,...,...,...,...,...,...,...
318972,06/02/2020,Non-Emergency Police Matter,Social Distancing,Store/Commercial,45-20 GREENPOINT AVENUE,SUNNYSIDE,QUEENS,40.741936,-73.919623
318261,06/02/2020,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,488 WEST 130 STREET,NEW YORK,MANHATTAN,40.815552,-73.954126
319569,06/02/2020,Noise - Residential,Loud Music/Party,Residential Building/House,95 OLD BROADWAY,NEW YORK,MANHATTAN,40.818146,-73.955374
318854,06/02/2020,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,37-37 83 STREET,JACKSON HEIGHTS,QUEENS,40.749505,-73.883310


## Updating Covid_Final_DF with total number of crimes for each Borough and Date

In [19]:
# Grouping DF by Borough and Date
total= nypd_final_df.groupby(["Borough","Date"])

crimecount = total["ComplaintType"].count()

crimecount = pd.DataFrame(crimecount)
crimecount = crimecount.reset_index()

# Merging Covid and Crimes by Borough and Date
covid_final_df = crimecount.merge(covid_final_df, on =("Borough", "Date"))

# Renaming the columns
covid_final_df = covid_final_df.rename(columns = {"ComplaintType": "TotalCrimes"})

covid_final_df

,Borough,Date,TotalCrimes,Cases,Hospitalizations,Deaths,Latitude,Longitude
0,BRONX,02/29/2020,300,0,4,0,40.83,-73.92
1,BRONX,03/01/2020,343,1,1,0,40.83,-73.92
2,BRONX,03/02/2020,249,0,9,0,40.83,-73.92
3,BRONX,03/03/2020,233,1,7,0,40.83,-73.92
4,BRONX,03/04/2020,243,0,6,0,40.83,-73.92
...,...,...,...,...,...,...,...,...
470,STATEN ISLAND,05/29/2020,99,15,0,1,40.58,-74.15
471,STATEN ISLAND,05/30/2020,151,9,0,0,40.58,-74.15
472,STATEN ISLAND,05/31/2020,140,6,0,1,40.58,-74.15
473,STATEN ISLAND,06/01/2020,73,2,0,0,40.58,-74.15


In [20]:
# Creating Total Number of Crimes based on date

crimes_total_df = nypd_final_df.groupby(['Date']).count()
crimes_total_df = crimes_total_df.reset_index()
crimes_total_df

,Date,ComplaintType,Descriptor,locationType,incidentAddress,City,Borough,Latitude,Longitude
0,02/29/2020,1831,1831,1831,1831,1831,1831,1831,1831
1,03/01/2020,1845,1845,1845,1845,1845,1845,1845,1845
2,03/02/2020,1470,1470,1470,1470,1470,1470,1470,1470
3,03/03/2020,1447,1447,1447,1447,1447,1447,1447,1447
4,03/04/2020,1405,1405,1405,1405,1405,1405,1405,1405
...,...,...,...,...,...,...,...,...,...
90,05/29/2020,3510,3510,3510,3510,3510,3510,3510,3510
91,05/30/2020,4976,4976,4976,4976,4976,4976,4976,4976
92,05/31/2020,4702,4702,4702,4702,4702,4702,4702,4702
93,06/01/2020,2258,2258,2258,2258,2258,2258,2258,2258


## Importing final DataFrames to SQL

In [21]:
crimes_total_df=crimes_total_df[['Date','ComplaintType']]
crimes_total_df

,Date,ComplaintType
0,02/29/2020,1831
1,03/01/2020,1845
2,03/02/2020,1470
3,03/03/2020,1447
4,03/04/2020,1405
...,...,...
90,05/29/2020,3510
91,05/30/2020,4976
92,05/31/2020,4702
93,06/01/2020,2258


In [22]:
total_info_map= covid_total_df.merge(crimes_total_df, on='Date')
total_info_map

,Date,TotalCases,TotalHospitalizations,TotalDeaths,ComplaintType
0,02/29/2020,1,12,0,1831
1,03/01/2020,1,4,0,1845
2,03/02/2020,0,22,0,1470
3,03/03/2020,2,19,0,1447
4,03/04/2020,5,21,0,1405
...,...,...,...,...,...
90,05/29/2020,423,85,28,3510
91,05/30/2020,229,43,28,4976
92,05/31/2020,137,27,8,4702
93,06/01/2020,121,13,7,2258


In [23]:
connection_string = f"postgres:{password}@localhost:5432/NYC_COVID19_CRIMES_DB"
engine = create_engine(f'postgresql://{connection_string}')

# Final DF for NYC-COVID-19
covid_final_df.to_sql(name='covid', con=engine, if_exists='replace', index=True)

# Final DF for NYC-CRIMES
nypd_final_df.to_sql(name='crime', con=engine, if_exists='replace', index=True)

# Final DF for total info map
total_info_map.to_sql(name='summary', con=engine, if_exists='replace', index=True)